# COA's to text by Python OCR with Machine Learning

First lets make a some commands to set up the virtual environment.
- python -m venv ocr-env
- .\ocr-env\Scripts\Activate.ps1

Now that the terminal is in the **ocr-env** virtual environment, lets install some packages that we can use...
- pip install pdf2image opencv-python notebook numpy pillow pytesseract

Linux dependencies for Ubuntu
- sudo apt install poppler-utils tesseract-ocr-eng python3-tesserocr -y

## Step 1: convert all *.pdf* files in the pdf folder to *.jpg* in jpg folder.

In [68]:
from pdf2image import convert_from_path
import os
import platform

In [ ]:
# Detect the os type
os_platform = platform.system()

In [69]:
# pdf 2 jpg function
def pdf2jpg(pdf_file_path, new_file_name, jpg_path):
    convert_from_path(pdf_path=pdf_file_path, output_folder=jpg_path, single_file=True, 
                      fmt='jpeg', output_file=new_file_name, grayscale=True, jpegopt={   
                          "quality": 100,
                          "progressive": True,
                          "optimize": True
                      })

In [70]:
current_path = os.getcwd()

jpg_path = current_path + "/jpg"
pdf_path = current_path + "/pdf"
txt_path = current_path + "/txt"

# convert all pdfs to 
for root, dir_name, file_name in os.walk(pdf_path):
    for name in file_name:
        new_file_name = name.replace(".pdf", "")
        print(name)
        pdf2jpg(os.path.join(root, name),new_file_name, jpg_path)

df
3001156  10_30.pdf
3001156 03_04.pdf
3001156 04-20.pdf
3001156 04_02.pdf
3001156 1-20 rw.pdf
3001156 1-6 sf.pdf
3001156 10-05.pdf
3001156 11-20.pdf
3001156 12-14.pdf
3001156 2-16 rw.pdf
3001156 3-27.pdf
3001156 4-15 rw.pdf
3001156 5-20 sf.pdf
3001156 5-5.pdf
3001156 6-1 ss.pdf
3001156 6-30.pdf
3001156 7-8 rw.pdf
3001156 803 rw.pdf
3001156 9-11.pdf
3001157  07_23.pdf
3001157 01-10.pdf
3001157 02-14.pdf
3001157 03-24.pdf
3001157 04-12.pdf
3001157 07-06.pdf
3001157 10-28 lg.pdf
3001157 10-8 ss.pdf
3001157 10-9 ss.pdf
3001157 11-17 lg.pdf
3001157 2-7 rw.pdf
3001157 4-9 ss.pdf
3001157 6-29 ss.pdf
3001157 8-3 rw.pdf
3001166  11_27.pdf
3001166 02_14.pdf
3001166 04_01.pdf
3001167 12-1 sf.pdf
3001167 2-16 rw.pdf
3001167 6-18 rw.pdf
3001168 5-5 rw.pdf
3001171 07-06.pdf
3001172  06_22.pdf
3001172 06-09.pdf
3001172 11-18.pdf
3001172 4-7 ss.pdf
3001175 01_05.pdf
3001175 02-02.pdf
3001175 03_03.pdf
3001175 03_05.pdf
3001175 04-23.pdf
3001175 05-29.pdf
3001175 1-3 ss.pdf
3001175 11-11.pdf
3001175 

## Step 2: deskewing text with openCV
We will use openCv and numpy to preprocess the images from above so the text is aligned and the colors inverted, 

In [71]:
import numpy as np
import cv2

In [72]:
# for root, dir_name, file_names in os.walk(jpg_path):
#     for name in file_names:
        
#         # read image into open cv
#         image_path = os.path.join(root, name)
#         image = cv2.imread(image_path)
        
#         # convert image to grayscale
#         gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        
#         # invert image colors (black -> white and white -> black)
#         gray = cv2.bitwise_not(gray)
        
#         # threshold the image, setting all forground pixels to 255 and backgroud pixels to 0
#         thresh = cv2.threshold(gray, 0, 255,
#                               cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
        
#         cv2.imshow("Input", gray)
#         cv2.waitKey(0)
        
#         # grab the (x,y) coordinates of all pixel values that are 
#         # greater than 0, then use these coordinates to compute
#         # a rotated bounding box that contains all coordinates
#         coords = np.column_stack(np.where(thresh > 0))
#         angle = cv2.minAreaRect(coords)[-1]
        
        
#         if abs(angle) > 45:
#             print(f"Image \"{name}\" has an angle of {angle}, setting to 0.")
#             angle = 0
        
#         # rotate and deskew the image
#         (h,w) = image.shape[:2]
#         center = (w // 2, h // 2)
#         M = cv2.getRotationMatrix2D(center, angle, 1.0)
#         rotated = cv2.warpAffine(image, M, (w, h),
#                                 flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
        
#         # draw the correction angle on the image so we can validate it
#         cv2.imshow("Input", image)
#         cv2.imshow("Rotated", rotated)
#         cv2.waitKey(0)

## Step 3: convert image text to a text file with Tesseract and python bindings


In [73]:
from PIL import Image
import pytesseract

In [74]:
for root, dir_name, file_names in os.walk(jpg_path):
    for name in file_names:
        
        # read image into open cv
        image_path = os.path.join(root, name)
        image = cv2.imread(image_path)
        
        # convert image to grayscale
        gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        
        # invert image colors (black -> white and white -> black)
        # gray = cv2.bitwise_not(gray)
        
        # threshold the image, setting all forground pixels to 255 and backgroud pixels to 0
        # thresh = cv2.threshold(gray, 0, 255,
        #                       cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
        # median blurring to remove noise
        #gray = cv2.medianBlur(gray, 3)
        
        print(f"Writing {name}...")
        # write the file to disk
        cv2.imwrite(os.path.join(root,name), gray)
        
        ocr_config = ' -l eng --psm 11 --oem 3 -c preserve_interword_spaces=1'

        # ## Text File conversion
       
        # text = pytesseract.image_to_string(Image.open(os.path.join(root,name)), config=ocr_config)
        # with open(txt_path + text_file_name, 'w+') as f:
        #     f.write(text)

        ## PDF File Conversion: converts a pdf into a readable (searchable) pdf
        text_file_name = "/" + name.replace(".jpg",".txt")
        hocr_file_name = "/" + name.replace(".jpg",".pdf")
        hocr = pytesseract.image_to_pdf_or_hocr(Image.open(os.path.join(root,name)), config=ocr_config)

        with open(txt_path + hocr_file_name, "w+b") as f:
            f.write(hocr)

print("Done...")

.
Writing 8000149  12_30 a.jpg...
Writing 8000149  12_30 b.jpg...
Writing 8000149  12_31.jpg...
Writing 8000149 01-22.jpg...
Writing 8000149 01_02.jpg...
Writing 8000149 02-02.jpg...
Writing 8000149 02-05.jpg...
Writing 8000149 02-09.jpg...
Writing 8000149 02-12.jpg...
Writing 8000149 02-13.jpg...
Writing 8000149 02-14.jpg...
Writing 8000149 02-23.jpg...
Writing 8000149 02-26.jpg...
Writing 8000149 02_05.jpg...
Writing 8000149 02_15.jpg...
Writing 8000149 02_16.jpg...
Writing 8000149 03-06 A.jpg...
Writing 8000149 03-06.jpg...
Writing 8000149 03-08.jpg...
Writing 8000149 03-09.jpg...
Writing 8000149 03-15.jpg...
Writing 8000149 03-31 b.jpg...
Writing 8000149 03-31.jpg...
Writing 8000149 03_13.jpg...
Writing 8000149 03_14.jpg...
Writing 8000149 04-01.jpg...
Writing 8000149 04-05.jpg...
Writing 8000149 04-20.jpg...
Writing 8000149 04_12.jpg...
Writing 8000149 05-10.jpg...
Writing 8000149 05-19.jpg...
Writing 8000149 05-21.jpg...
Writing 8000149 05-22 a.jpg...
Writing 8000149 05-22.jpg...